In [1]:
import pandas as pd
import numpy as np
from math import pow, sqrt

## Read the books file to grab the book-title

In [2]:
books = pd.read_csv('BX-Books.csv',sep=';',names=['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Image-URL-S','Image-URL-M','Image-URL-L'])

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def read_file_into_dictionary():
    data = {}
    i = 0
    f = open("BX-Book-Ratings.csv", encoding='utf8')
    for line in f:
        i += 1
        fields = line.split(';')
        user = fields[0].strip('"')
        book = fields[1].strip('"')
        rating = int(fields[2].strip().strip('"'))
        if user in data:
            currentRatings = data[user]
        else:
            currentRatings = {}
        currentRatings[book] = rating
        data[user] = currentRatings
    return data
    f.close()

In [4]:
ratings = read_file_into_dictionary()

## Data is sparse, Cosine Similarity

In [5]:
def Cosine(ratings1, ratings2):
    sum_xy= 0
    sum_x_sqr = 0
    sum_y_sqr = 0
    for key in ratings1:
        x = ratings1[key]
        sum_x_sqr += pow(x,2)
        if key in ratings2:
            x = ratings1[key]
            y = ratings2[key]
            sum_xy += x*y
    for key in ratings2:
        y = ratings2[key]
        sum_y_sqr += pow(y,2)
    if sqrt((sum_x_sqr*sum_y_sqr)) !=0:
        return sum_xy/sqrt((sum_x_sqr*sum_y_sqr))
    else:
        return 0

In [6]:
def NearestNeighbor_Cosine(username, dataset):
    distances=[]
    for user in dataset:
        if user != username:
            distances.append((user,Cosine(dataset[user],dataset[username])))
    distances.sort(key = lambda i: i[1],reverse=True)
    return distances

In [7]:
def recommend(username, dataset, n_of_neighbor):
    recommendations={}
    total = 0
    nearestneighbor = NearestNeighbor_Cosine(username,dataset)
    userRatings = dataset[username]
    for i in range(n_of_neighbor):
        total += nearestneighbor[i][1]
    if total !=0:
        for i in range(n_of_neighbor):
            weight = nearestneighbor[i][1]/total
            name = nearestneighbor[i][0]
            neighborRatings = dataset[name]
            for artist in neighborRatings:
                if artist not in userRatings:
                    if artist not in recommendations:
                        recommendations[artist] = (neighborRatings[artist] * weight)
                    else:
                        recommendations[artist] = (recommendations[artist] + neighborRatings[artist]*weight)
    
    # Convert ISBN into book-title
    df = pd.DataFrame([[key,value] for key,value in recommendations.items()],columns=["ISBN","Recommend_rating"])
    df = pd.merge(df,books,how='left',on='ISBN').sort_values(by=['Recommend_rating'],ascending=False)['Book-Title']
    df.index = np.arange(1, len(df) + 1)
    if len(df) > 10:
        df = df[:10]
    return df

In [8]:
# the third number is the # of neighbour to include in the calculation
recommend('276747',ratings,20)

1       Divine Secrets of the Ya-Ya Sisterhood: A Novel
2                               Standing in the Rainbow
3                                      Sex and the City
4                        Mommie Dearest : Joan Crawford
5                 Heal Thyself for Health and Longevity
6     Kitchen Confidential: Adventures in the Culina...
7                                    The Night Listener
8                                     Portrait in Death
9                                        Sophies Choice
10                                            Outlander
Name: Book-Title, dtype: object